<a href="https://colab.research.google.com/github/abdurrazzz/OIBSIP/blob/main/emailspam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB

In [13]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

In [14]:
url = "https://github.com/mohitgupta-1O1/Kaggle-SMS-Spam-Collection-Dataset-/raw/master/spam.csv"
email = pd.read_csv(url, encoding="ISO-8859-1")

In [15]:
email.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [16]:
label_encoder = LabelEncoder()
email.v1 = label_encoder.fit_transform(email.v1)
print(email.head())

   v1                                                 v2 Unnamed: 2  \
0   0  Go until jurong point, crazy.. Available only ...        NaN   
1   0                      Ok lar... Joking wif u oni...        NaN   
2   1  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   0  U dun say so early hor... U c already then say...        NaN   
4   0  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [17]:
email['v1'].value_counts()

0    4825
1     747
Name: v1, dtype: int64

In [18]:
email.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   int64 
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: int64(1), object(4)
memory usage: 217.8+ KB


In [19]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import string
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
corpus = []
ps = PorterStemmer()
for i in range(len(email)):

    msg = email['v2'][i]
    msg = re.sub(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', msg)
    msg = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr', msg)
    msg = re.sub(r'£|\$', 'moneysymb', msg)
    msg = re.sub(r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', 'phonenumbr', msg)
    msg = re.sub(r'\d+(\.\d+)?', 'numbr', msg)

    msg = re.sub(r'[^\w\d\s]', ' ', msg)

    msg = msg.lower()

    msg = msg.split()

    msg = [ps.stem(word) for word in msg if word not in set(stopwords.words('english'))]

    msg = ' '.join(msg)

    corpus.append(msg)

    # Print the preprocessed message for the first two iterations
    if i < 2:
        print("\t\t\t\t MESSAGE ", i)
        print("\n Final Prepared - Message ", i, " : ", msg, "\n\n")

# Print only the first twenty messages in the corpus
print("First Twenty Messages of the Preprocessed Corpus:")
for i in range(min(20, len(corpus))):
    print(f"Message {i}: {corpus[i]}")

				 MESSAGE  0

 Final Prepared - Message  0  :  go jurong point crazi avail bugi n great world la e buffet cine got amor wat 


				 MESSAGE  1

 Final Prepared - Message  1  :  ok lar joke wif u oni 


First Twenty Messages of the Preprocessed Corpus:
Message 0: go jurong point crazi avail bugi n great world la e buffet cine got amor wat
Message 1: ok lar joke wif u oni
Message 2: free entri numbr wkli comp win fa cup final tkt numbrst may numbr text fa numbr receiv entri question std txt rate c appli numbrovernumbr
Message 3: u dun say earli hor u c alreadi say
Message 4: nah think goe usf live around though
Message 5: freemsg hey darl numbr week word back like fun still tb ok xxx std chg send åmoneysymbnumbr rcv
Message 6: even brother like speak treat like aid patent
Message 7: per request mell mell oru minnaminungint nurungu vettam set callertun caller press numbr copi friend callertun
Message 8: winner valu network custom select receivea åmoneysymbnumbr prize reward claim call 

In [21]:
cv = CountVectorizer()
x = cv.fit_transform(corpus).toarray()
y = email['v1']
print (y.value_counts())

print(y[0])
print(y[1])

0    4825
1     747
Name: v1, dtype: int64
0
0


In [22]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y,test_size= 0.20, random_state = 0)
bayes_classifier = GaussianNB()
bayes_classifier.fit(xtrain, ytrain)

GaussianNB()

In [23]:
y_pred = bayes_classifier.predict(xtest)
cm = confusion_matrix(ytest, y_pred)
cm
print ("Accuracy : %0.5f \n\n" % accuracy_score(ytest, bayes_classifier.predict(xtest)))
print (classification_report(ytest, bayes_classifier.predict(xtest)))

Accuracy : 0.86368 


              precision    recall  f1-score   support

           0       0.97      0.87      0.92       949
           1       0.53      0.85      0.65       166

    accuracy                           0.86      1115
   macro avg       0.75      0.86      0.78      1115
weighted avg       0.90      0.86      0.88      1115

